In [0]:
dbutils.widgets.dropdown(name="gams_version",  defaultValue="latest", choices=["latest"] + [str(i) for i in range(49, 41, -1)], label="GAMS Version")
gams_version = dbutils.widgets.get("gams_version")

gams_dir = "/dbfs/GAMS"
gams_dir = "/Volumes/ea_dev/sourcefiles/datalake/GAMS"

In [0]:
%pip install bs4

In [0]:
import hashlib
import requests
import os
from bs4 import BeautifulSoup
import zipfile
import re

In [0]:
def GetGAMSLinuxInstall(version = 0):
    # This defines the URL of the webpage that the script will request.
    if version == 0:
        url = "https://www.gams.com/download/"
    else:
        url = f"https://www.gams.com/{version}/"
    
    # Sends an HTTP GET request to fetch the content of the webpage. The response contains the HTML source code of the page.
    response = requests.get(url)

    # Raises an error (HTTPError) if the request fails (e.g., 404 Not Found, 500 Server Error).
    response.raise_for_status()

    # Parses the HTML content of the webpage using BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')

    # Finds all <button> elements on the page. The result is a list of <button> elements found in the HTML.
    buttons = soup.find_all('button')

    # This snippet extracts the first button that contains a data-href attribute with "linux" in its URL.
    button_link = [button['data-href'] for button in buttons if 'data-href' in button.attrs and 'linux' in button['data-href'].lower()]
    button_link = button_link[0]
    
    return button_link  

def download_file(url, local_path):
    # Ensure the directory exists
    os.makedirs(os.path.dirname(local_path), exist_ok=True)

    # Download the file
    response = requests.get(url)
    response.raise_for_status()
    
    # Save the file locally
    with open(local_path, 'wb') as file:
        file.write(response.content)
    
    print(f"File downloaded: {local_path}")

def unzip_file(file_path, extract_to): # this function is used to unzip the downloaded file
    with zipfile.ZipFile(file_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)
    print(f"File unzipped to: {extract_to}")

In [0]:
url = GetGAMSLinuxInstall(version=gams_version)
gamsversion = re.search(r'/distributions/([^/]+)/linux', url)
local_path = f"{gams_dir}/gamsDownload.exe"
download_file(url, local_path)


In [0]:
# unzip/install GAM to this "/dbfs/GAMS" folder
unzip_file(file_path=local_path, extract_to=gams_dir)

In [0]:
for f in os.listdir(gams_dir):
  if f.startswith('gams'):
    print(f)
    dbutils.fs.cp('/Volumes/ea_dev/sourcefiles/datalake/GAMS/gamslice.txt', f'{gams_dir}/{f}/'.replace('/dbfs', 'dbfs:'))

In [0]:
dbutils.fs.rm(f"{gams_dir.replace('/dbfs', 'dbfs:')}/gamsDownload.exe")